In [2]:
import sys
sys.path.append("../src")
import paths
import dataset
import train
import utils
import torch
import pickle
import evaluation

from datasets import load_from_disk

/home/user/mnlp/notebooks/../src/paths.py


/home/user/mnlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train = False

In [4]:
TRAIN_DATA = paths.data/"manzoni_train_tokens.csv"
OOD_DATA = paths.data/"manzoni_dev_tokens.csv"
HF_DATA = paths.data/"prepared"
torch.set_float32_matmul_precision("high")   # enable TF32 matmuls on Ampere
torch.backends.cudnn.allow_tf32 = True 

In [5]:
import numpy as np
from collections import Counter

def check_labels(hfds_split, sample_rows=2000):
    # Concatenate labels from a subset of rows (pad to same length already handled by collator)
    n = min(sample_rows, len(hfds_split))
    cats = []
    for ex in hfds_split.select(range(n)):
        labs = np.array(ex["labels"])
        cats.append(labs)
    all_labs = np.concatenate(cats)
    visible = all_labs[all_labs != -100]
    uniq = np.unique(visible)
    print("Unique visible labels:", uniq)
    bad = [x for x in uniq if x not in (0, 1)]
    if bad:
        print("❌ Found out-of-range labels:", bad)
    else:
        print("✅ Labels look fine (only 0/1).")
    return uniq

In [6]:
if train:
    import importlib
    importlib.reload(dataset)
    importlib.reload(train)

    import os
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # makes the exception point to the correct op

    results = {}
    for model_key in ["deberta", "modernbert", "bert"]:
        pairs = dataset.read_token_label_file(TRAIN_DATA)
        sents_tok, sents_lab = dataset.group_into_sentences(pairs)
        ds_full = dataset.build_hf_dataset_for_token_classification(sents_tok, sents_lab, model_key=model_key)
        split = ds_full.train_test_split(train_size=0.8, seed=69)
        train_ds = dataset.tidy(split["train"], model_key)
        val_ds   = dataset.tidy(split["test"], model_key)
        train_ds.save_to_disk(HF_DATA/f"{model_key}"/"train")
        val_ds.save_to_disk(HF_DATA/f"{model_key}"/"val")
        _ = check_labels(train_ds)  # your ModernBERT train split
        _ = check_labels(val_ds)
        print(f"\n=== Training {model_key} -> {utils.MODEL_SPECS[model_key].name} ===")
        out_dir = str(paths.chekpoints / model_key)
        results[model_key] = train.train_token_splitter(
            train_ds, val_ds,
            model_key=model_key, out_dir=out_dir,
            lr=5e-5, batch_size=8, epochs=3,
        )
    with open(paths.results/"token_class_eval.pkl", "wb") as f:
        pickle.dump(results, f)
else:
    with open(paths.results/"token_class_eval.pkl", "rb") as f:
        results = pickle.load(f)

print(results)

{'deberta': {'eval_loss': 0.00109088362660259, 'eval_precision': 0.9898477157360406, 'eval_recall': 0.9982935153583617, 'eval_f1': 0.994052676295667, 'eval_accuracy': 0.9995906672124437, 'eval_runtime': 1.1752, 'eval_samples_per_second': 50.205, 'eval_steps_per_second': 6.807, 'epoch': 3.0}, 'modernbert': {'eval_loss': 0.007550484966486692, 'eval_precision': 0.976271186440678, 'eval_recall': 0.9473684210526315, 'eval_f1': 0.9616026711185309, 'eval_accuracy': 0.9972119522395296, 'eval_runtime': 0.8784, 'eval_samples_per_second': 15.939, 'eval_steps_per_second': 2.277, 'epoch': 3.0}, 'bert': {'eval_loss': 0.002381410449743271, 'eval_precision': 0.993127147766323, 'eval_recall': 0.9897260273972602, 'eval_f1': 0.9914236706689536, 'eval_accuracy': 0.9994210952877156, 'eval_runtime': 0.5338, 'eval_samples_per_second': 99.281, 'eval_steps_per_second': 13.113, 'epoch': 3.0}}


In [7]:
import pandas as pd
pd.DataFrame(results).T.sort_values("eval_f1", ascending=False)

,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
deberta,0.001091,0.989848,0.998294,0.994053,0.999591,1.1752,50.205,6.807,3.0
bert,0.002381,0.993127,0.989726,0.991424,0.999421,0.5338,99.281,13.113,3.0
modernbert,0.007550,0.976271,0.947368,0.961603,0.997212,0.8784,15.939,2.277,3.0


In [20]:
import importlib
importlib.reload(evaluation)
best_key = max(results, key=lambda k: results[k]["eval_f1"])
model_dir = paths.chekpoints/best_key
best_trainer = evaluation.load_trainer_for_eval(model_dir, HF_DATA/best_key/"val")
val_ds = load_from_disk(HF_DATA/best_key/"val")

pred = best_trainer.predict(val_ds)  # logits + label_ids as np arrays
logits = pred.predictions
label_ids = pred.label_ids
tok = best_trainer.tokenizer

def sentences_from_word_seq(words, y_pred):
    sents, cur = [], []
    for w, b in zip(words, y_pred):
        cur.append(w)
        if b == 1:
            sents.append(cur); cur = []
    if cur: sents.append(cur)
    return sents

for i in range(min(3, len(val_ds))):
    ids = val_ds[i]["input_ids"]
    words = tok.convert_ids_to_tokens(ids)

    mask = (label_ids[i] != -100)          # np.bool_ array
    y_pred = logits[i].argmax(-1)[mask]    # predicted boundary labels at visible positions
    visible_words = [w for w, m in zip(words, mask.tolist()) if m]

    sents = sentences_from_word_seq(visible_words, y_pred)
    print(f"\nWindow {i} — predicted {len(sents)} sentences:")
    print(" | ".join([" ".join(s) for s in sents]))


evaluation.preview_predictions(best_trainer, val_ds, k=3)
evaluation.preview_full_sentences(best_trainer, val_ds, n_examples=2)
evaluation.preview_pred_vs_gold(best_trainer, val_ds, [1,2,3])

/home/user/mnlp/notebooks/../src/evaluation.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



Window 0 — predicted 11 sentences:
"," ▁il ▁ ▁si ▁moss ▁ ▁Gert ▁ ▁la ▁princip ▁ ▁il ▁principi ▁lo ▁seguir ▁ ▁sce ▁tutti ▁le ▁scale ▁ ▁ ▁monta ▁in ▁carro ▁ | ▁Gl ▁imp ▁ ▁le ▁noi ▁del ▁mondo ▁ ▁ ▁la ▁vita ▁beat ▁del ▁chi ▁ ▁principal ▁per ▁le ▁ ▁di ▁sangu ▁ ▁ ▁fur ▁il ▁tema ▁ ▁conversa ▁ ▁d ▁il ▁tragi ▁ | ▁Sul ▁ ▁ ▁strada ▁ ▁il ▁ ▁rinn ▁ ▁ ▁alla ▁ ▁ ▁ ▁le ▁ripet ▁p ▁ ▁la ▁form ▁ ▁ri ▁ | ▁All ▁entra ▁in ▁Mon ▁ ▁Gert ▁si ▁sent ▁string ▁il ▁cu ▁ ▁ma ▁la ▁sua ▁at ▁fu ▁att ▁per ▁un ▁ist ▁da ▁non ▁so ▁quali ▁ ▁che ▁ ▁ ▁ferma ▁la ▁carro ▁ ▁recita ▁non ▁so ▁qual ▁compliment ▁ | ▁Ri ▁il ▁cam ▁ ▁ ▁and ▁quasi ▁di ▁ ▁al ▁monast ▁ ▁tra ▁gli ▁ ▁de ▁curios ▁ ▁che ▁accor ▁da ▁ ▁le ▁parti ▁ ▁strada ▁ | ▁Al ▁ferm ▁ ▁carro ▁ ▁d ▁ ▁ ▁mura ▁ ▁d ▁ ▁quell ▁porta ▁ ▁il ▁cu ▁si ▁stri ▁ ▁p ▁ ▁Gert ▁ | ▁Si ▁ ▁tra ▁due ▁ale ▁di ▁popol ▁ ▁che ▁ ▁servi ▁face ▁stare ▁indie ▁ | ▁ ▁que ▁ ▁add ▁alla ▁pover ▁ ▁ ▁ ▁studi ▁continu ▁il ▁suo ▁conte ▁ ▁ma ▁p ▁di ▁tutti ▁quell ▁in ▁ ▁la ▁ten ▁in ▁sugg ▁ ▁due ▁del ▁padre ▁ ▁ ▁q

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



Example 0 — 8 predicted sentences:
 • "," il si moss Gert la princip il principi lo seguir sce tutti le scale monta in carro Gl imp le noi del mondo la vita
 • beat del chi principal per le di sangu fur il tema conversa d il tragi Sul strada il rinn alla le ripet p la form ri All entra in Mon Gert si sent
 • string il cu ma la sua at fu att per un ist da non so quali che ferma la carro recita non so
 • qual compliment Ri il cam and quasi di al monast tra gli de curios che accor da le parti strada Al ferm carro d mura d quell porta il cu si stri p Gert Si tra due ale di
 • popol che servi face stare indie que add alla pover studi continu il suo conte ma p di tutti quell in la ten in sugg due del
 • padre quali essa quant ne av cos gran pa non po di suo momento E que govern le su moss il suo come per
 • m di red in Attra il primo corti in un si vide la porta del
 • chi interno spal occupa da mona Nell prima fila la ba circo da

Example 1 — 12 predicted sentences:
 • evano ces di neve sp

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



### Example 0 — 11 predicted sentences

 • "," il principe si mosse ; Gertrude "," la principessa e il principino lo seguirono ; scesero tutti le scale "," e montarono in carrozza .
 • Gl' impicci e le noie del mondo "," e la vita beata del chiostro "," principalmente per le giovani di sangue nobilissimo "," furono il tema della conversazione "," durante il tragitto .
 • Sul finir della strada "," il principe rinnovò l' istruzioni alla figlia "," e le ripeté più volte la formola della risposta .
 • All' entrare in Monza "," Gertrude si sentì stringere il cuore ; ma la sua attenzione fu attirata per un istante da non so quali signori che "," fatta fermar la carrozza "," recitarono non so qual complimento .
 • Ripreso il cammino "," s' andò quasi di passo al monastero "," tra gli sguardi de' curiosi "," che accorrevano da tutte le parti sulla strada .
 • Al fermarsi della carrozza "," davanti a quelle mura "," davanti a quella porta "," il cuore si strinse ancor più a Gertrude .
 • Si s


### Window 1
P: evano assomigliarsi a cespugli coperti di neve "," sporgenti da un dirupo "," al chiaro di luna . | « Ah ! ah ! » fu il suo saluto "," mentre si levava gli occhiali "," e li riponeva nel libricciolo . | « Dirà il signor curato "," che son venuto tardi "," » disse Tonio "," inchinandosi "," come pure fece "," ma più goffamente "," Gervaso . | « Sicuro ch' è tardi : tardi in tutte le maniere . | Lo sapete "," che sono ammalato ? » « | Oh ! mi dispiace . » « | L' avrete sentito dire ; sono ammalato "," e non so quando potrò lasciarmi vedere ... Ma | perché vi siete condotto dietro quel ... quel figliuolo ? » « | Così per compagnia "," signor curato . » « | Basta "," vediamo . » « | Son venticinque berlinghe nuove "," di quelle col sant' Ambrogio a cavallo "," » disse Tonio "," levandosi un involtino di tasca . | « Vediamo "," » replicò don Abbondio : e "," preso l' involtino "," si rimesse gli occhiali "," l' aprì "," cavò le berlinghe "," le contò "," le voltò "," le riv

In [21]:
def error_examples(trainer, ds, max_show=10):
    out = trainer.predict(ds)
    preds = out.predictions.argmax(-1)
    labels = out.label_ids
    mask = labels != -100
    ids = ds["input_ids"]
    tok = trainer.tokenizer
    shown = 0
    results = []
    for i in range(len(ds)):
        m = mask[i]
        if not m.any(): continue
        y_true = labels[i][m]
        y_pred = preds[i][m]
        if (y_true != y_pred).any():
            results.append(i)
            words = tok.convert_ids_to_tokens(ds[i]["input_ids"])
            visible_words = [w for w,mm in zip(words, m) if mm]
            # mark predicted boundaries with "▌"
            pieces = []
            for w, b, t in zip(visible_words, y_pred, y_true):
                mark = "▌" if b==1 else ""
                pieces.append(w+mark)
            print(" ".join(pieces))
            shown += 1
            if shown >= max_show: break
    return results

results = error_examples(best_trainer, val_ds, max_show=5)
evaluation.preview_pred_vs_gold(best_trainer, val_ds, results)



Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


▁» ▁ ▁qui ▁una ▁buon ▁bestemm ▁ ▁« ▁ ▁chi ▁lo ▁far ▁non ▁se ▁ne ▁pent ▁ ▁per ▁non ▁ne ▁av ▁tempo ▁ ▁ ▁... ▁» ▁un ▁ ▁bestemm ▁▌ ▁« ▁Zit ▁ ▁zit ▁ ▁» ▁ri ▁il ▁primo ▁or ▁ ▁« ▁il ▁ ▁curat ▁ ▁un ▁ ▁che ▁sa ▁il ▁ ▁del ▁mondo ▁ ▁ ▁noi ▁si ▁gal ▁ ▁che ▁non ▁v ▁far ▁del ▁male ▁ ▁pur ▁ ▁ ▁▌ ▁Sign ▁curat ▁ ▁ ▁illustr ▁ ▁don ▁Rodrig ▁ ▁ ▁la ▁river ▁car ▁ ▁»▌ ▁ ▁nome ▁fu ▁ ▁ ▁ ▁di ▁don ▁Abb ▁ ▁come ▁ ▁nel ▁forte ▁d ▁un ▁temporal ▁not ▁ ▁un ▁lamp ▁che ▁illumina ▁moment ▁ ▁in ▁confus ▁gli ▁ ▁ ▁ ▁ac ▁il ▁terror ▁▌ ▁Fe ▁ ▁come ▁per ▁ist ▁ ▁un ▁grand ▁in ▁ ▁ ▁disse ▁ ▁« ▁se ▁mi ▁sape ▁sugger ▁... ▁»▌ ▁« ▁Oh ▁ ▁sugger ▁ ▁lei ▁che ▁sa ▁di ▁latino ▁ ▁» ▁interrup ▁ ▁il ▁ ▁ ▁con ▁un ▁ris ▁tra ▁lo ▁ ▁ ▁il ▁fero ▁▌ ▁« ▁A ▁lei ▁to ▁▌ ▁E ▁sopra ▁ ▁ ▁non ▁si ▁ ▁ ▁parola ▁su ▁ ▁av ▁ ▁che ▁le ▁ ▁dato ▁per ▁suo ▁bene ▁ ▁alt ▁... ▁ ▁... ▁sa ▁lo ▁ ▁che ▁fare ▁quel ▁tal ▁ ▁▌ ▁Via ▁ ▁che ▁vuo ▁che ▁si ▁dica ▁in ▁suo ▁nome ▁all ▁illustr ▁ ▁don ▁Rodrig ▁ ▁»▌ ▁« ▁Il ▁mio ▁ ▁... ▁»▌ ▁« ▁Si ▁spieg ▁me ▁ ▁»▌ ▁« ▁...▌ ▁Dis ▁..


### Window 18
P: » e qui una buona bestemmia "," « o chi lo farà non se ne pentirà "," perché non ne avrà tempo "," e ... » un' altra bestemmia . | « Zitto "," zitto "," » riprese il primo oratore "," « il signor curato è un uomo che sa il viver del mondo ; e noi siam galantuomini "," che non vogliam fargli del male "," purché abbia giudizio . | Signor curato "," l' illustrissimo signor don Rodrigo nostro padrone la riverisce caramente . » | Questo nome fu "," nella mente di don Abbondio "," come "," nel forte d' un temporale notturno "," un lampo che illumina momentaneamente e in confuso gli oggetti "," e accresce il terrore . | Fece "," come per istinto "," un grand' inchino "," e disse : « se mi sapessero suggerire ... » « | Oh ! suggerire a lei che sa di latino ! » interruppe ancora il bravo "," con un riso tra lo sguaiato e il feroce . | « A lei tocca . | E sopra tutto "," non si lasci uscir parola su questo avviso "," che le abbiam dato per suo bene ; altrimenti ... ehm ... sare

In [15]:
# OOD
ood_results = {}
importlib.reload(dataset)

# -- OOD evaluation loop:
ood_results = {}
for model_key in ["deberta", "modernbert", "bert"]:
    model_dir = paths.chekpoints / model_key
    trainer = evaluation.load_trainer_for_eval(model_dir, HF_DATA / model_key / "ood")
    pred = trainer.predict(trainer.eval_dataset)  # use their own OOD eval set
    logits = pred.predictions
    labels = pred.label_ids

    metrics = evaluation.compute_prf(logits, labels)
    ood_results[model_key] = metrics
    print(f"{model_key} OOD Results:", metrics)

/home/user/mnlp/notebooks/../src/evaluation.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


deberta OOD Results: {'precision': 0.1875, 'recall': 0.008379888268156424, 'f1': 0.016042780748663103, 'accuracy': 0.9654622243078367}


/home/user/mnlp/notebooks/../src/evaluation.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
W0808 20:39:03.129000 3279 torch/_inductor/utils.py:1436] [1/0_1] Not enough SMs to use max_autotune_gemm mode


modernbert OOD Results: {'precision': 0.9793510324483776, 'recall': 0.996996996996997, 'f1': 0.9880952380952381, 'accuracy': 0.9991709844559585}


/home/user/mnlp/notebooks/../src/evaluation.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


bert OOD Results: {'precision': 0.9920844327176781, 'recall': 0.9920844327176781, 'f1': 0.9920844327176781, 'accuracy': 0.9994380443944928}


In [17]:
pd.DataFrame(ood_results).T.sort_values("f1", ascending=False)

,precision,recall,f1,accuracy
bert,0.992084,0.992084,0.992084,0.999438
modernbert,0.979351,0.996997,0.988095,0.999171
deberta,0.187500,0.008380,0.016043,0.965462


In [25]:
best_key = max(ood_results, key=lambda k: ood_results[k]["f1"])
model_dir = paths.chekpoints/best_key

ood_best_trainer = evaluation.load_trainer_for_eval(model_dir, HF_DATA / best_key / "ood")
ood_ds = load_from_disk(HF_DATA/best_key/"val")
results = error_examples(ood_best_trainer, ood_ds, max_show=5)

evaluation.preview_predictions(ood_best_trainer, ood_ds, k=3)
evaluation.preview_full_sentences(ood_best_trainer, ood_ds, n_examples=2)
evaluation.preview_pred_vs_gold(ood_best_trainer, ood_ds, results)

/home/user/mnlp/notebooks/../src/evaluation.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


perder la testa ; ma " per ciò che riguarda quel seme tanto per de bra " certo è che esso continua a ger " il 22 settembre dell anno 1612 .▌ In quel giorno l Ill ed E Signore " il Sign Don Giovanni de Men " Marche de la Hy " Gent etc Govern etc " pen seriam ad esti .▌ A quest effetto " sp a Pan e Marco Tu Mala " stampa regi camera " la soli grid " corre ed ac " perché la stampa ad este de bra .▌ Ma questi visse ancora per riceve " il 24 de dell anno 1618 " gli stessi e più forti colpi dall Ill ed E Signore " il Sign don Gomez Sua de Figueroa " Duca di Feria " etc Govern etc Però non essendo essi morti ne di quelli " l Ill ed E Signore " il Sign Gonzalo Fernandez di Cor " sotto il cui governo ac la passe di don Ab " s era trovato costretto a rico e ri la soli grid contro i bra " il giorno 5 ottobre del 1627 " cioè un anno " un mese e due giorni prima di quel me av .▌ Né fu questa l ultima pubblicazione ; ma noi delle posterior non c dove far men " come di cosa che esce dal periodo della

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



Example 0 — 7 predicted sentences:
 • in né im punto dagli anni ; l occhio grave e viva " la fronte ser e pensiero ; con la can " nel pal " tra i seg dell asti " della med " della fat " una specie di flor ver : tutte le forme del volto indica che in altre età " c era stata quella che più propria si chiama belle ; l ab de pen solen e bene " la pace interna d una lunga vita " l amore degli uomini " la gi continua d una sp in " vi avevano sos una " dire quasi " belle seni " che sp an più in quella mag sem della por. Ten
 • anche lui " qualche momento " fi nell aspetto dell inn il suo sg pen " ed ese da lungo tempo a rit dai sem i pen ; e " sotto a quel fos e a quel tur " pare di s sempre più qualcosa di conforme alla sp da lui con al primo ann d una tal visita " tu anima " « o! » disse : « che pre visita è questa! e quanto vi de esse gra d una sì buona ri ; quant per me abbia un po del ri! » « Rim! » es
 • il signore mara " ma rad da quelle parole e da quel fare " e content che il cardin

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



### Example 0 — 8 predicted sentences

 • incurvato né impigrito punto dagli anni ; l ' occhio grave e vivace ", " la fronte serena e pensierosa ; con la canizie ", " nel pallore ", " tra i segni dell ' astinenza ", " della meditazione ", " della fatica ", " una specie di floridezza verginale : tutte le forme del volto indicavano che in altre età ", " c ' era stata quella che più propriamente si chiama bellezza ; l ' abitudine de ' pensieri solenni e benevoli ", " la pace interna d ' una lunga vita ", " l ' amore degli uomini ", " la gioia continua d ' una speranza ineffabile ", " vi avevano sostituita una ", " direi quasi ", " bellezza senile ", " che spiccava ancor più in quella magnifica semplicità della porpora. Ten
 • ##ne anche lui ", " qualche momento ", " fisso nell ' aspetto dell ' innominato il suo sguardo penetrante ", " ed esercitato da lungo tempo a ritrarre dai sembianti i pensieri ; e ", " sotto a quel fosco e a quel turbato ", " parendogli di scoprire sempre più qualco


### Window 14
P: perder la testa ; ma ", " per ciò che riguarda quel seme tanto pernizioso de ' bravi ", " certo è che esso continuava a germogliare ", " il 22 settembre dell ' anno 1612. In | quel giorno l ' Illustrissimo ed Eccellentissimo Signore ", " il Signor Don Giovanni de Mendozza ", " Marchese de la Hynojosa ", " Gentiluomo etc. Governatore etc. ", " pensò seriamente ad estirparlo. A | quest ' effetto ", " spedì a Pandolfo e Marco Tullio Malatesti ", " stampatori regii camerali ", " la solita grida ", " corretta ed accresciuta ", " perché la stampassero ad esterminio de ' bravi. Ma | questi vissero ancora per ricevere ", " il 24 decembre dell ' anno 1618 ", " gli stessi e più forti colpi dall ' Illustrissimo ed Eccellentissimo Signore ", " il Signor don Gomez Suarez de Figueroa ", " Duca di Feria ", " etc. Governatore etc. Però non essendo essi morti neppur di quelli ", " l ' Illustrissimo ed Eccellentissimo Signore ", " il Signor Gonzalo Fernandez di Cordova ", " sotto il cu